In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd

import sys
sys.path.append("./missing_data")

from mpl_toolkits.basemap import Basemap
from build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile
from build_dataset import split_into_tiles_subfolders_and_track_dates


In [2]:
# BASEMAP 
latcorners = [30, 48]
loncorners = [-7, 46]
basemap_obj = Basemap(
    projection='geos',
    rsphere=(6378137.0, 6356752.3142),
    resolution='i',
    area_thresh=10000.,
    lon_0=9.5,
    satellite_height=3.5785831E7
)
x_center, y_center = basemap_obj(9.5, 0)
Xmin, Ymin, px_scale_x, px_scale_y = compute_pixel_scale(basemap_obj, latcorners, loncorners)

In [3]:
1/px_scale_y/1000 * 224, 1/px_scale_x/1000 * 224

(757.5205385725667, 804.8615703175508)

In [24]:
# Percorso alla cartella dei frame
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised" 
#os.makedirs(output_dir, exist_ok=True)

filenames = get_files_from_folder(folder=input_dir, extension="png")
#print(f"Trovati {len(filenames)} files")
file_metadata = []  # contiene i nomi e le date
for fname in filenames:
    start_dt = extract_dates_pattern_airmass_rgb_20200101_0000(fname.name)
    file_metadata.append((fname, start_dt))

sorted_metadata = sorted(file_metadata, key=lambda x: x[1])  # Ordina per start_dt
#random_fnames =  [item[0] for item in file_metadata]
sorted_filenames = [item[0] for item in sorted_metadata]
print(f" Ci sono {len(sorted_filenames)} files.")

 Ci sono 117764 files.


In [25]:
sorted_metadata = sorted_metadata[:32]
sorted_metadata

[(PosixPath('../fromgcloud/airmass_rgb_20111101_0000.png'),
  datetime.datetime(2011, 11, 1, 0, 0)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0005.png'),
  datetime.datetime(2011, 11, 1, 0, 5)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0010.png'),
  datetime.datetime(2011, 11, 1, 0, 10)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0015.png'),
  datetime.datetime(2011, 11, 1, 0, 15)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0020.png'),
  datetime.datetime(2011, 11, 1, 0, 20)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0025.png'),
  datetime.datetime(2011, 11, 1, 0, 25)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0030.png'),
  datetime.datetime(2011, 11, 1, 0, 30)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0035.png'),
  datetime.datetime(2011, 11, 1, 0, 35)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0040.png'),
  datetime.datetime(2011, 11, 1, 0, 40)),
 (PosixPath('../fromgcloud/airmass_rgb_20111101_0045.png'),
  datetime.datetime(2011

In [26]:
#sf = [(p, dt) for p, dt in zip(sorted_filenames, [f[1] for f in sorted_files])]
#print(f"sorted filenames, len = {len(sf)}")
subfolder_info = split_into_tiles_subfolders_and_track_dates(sorted_filenames=sorted_metadata, output_dir=output_dir)
print(f"Creati {len(subfolder_info)} tile-video in total.")

num_subfolders: 2
0 tilex 0 	 tiley 0 part_index 2
1 tilex 112 	 tiley 0 part_index 2
2 tilex 224 	 tiley 0 part_index 2
3 tilex 336 	 tiley 0 part_index 2
4 tilex 448 	 tiley 0 part_index 2
5 tilex 560 	 tiley 0 part_index 2
6 tilex 672 	 tiley 0 part_index 2
7 tilex 784 	 tiley 0 part_index 2
8 tilex 896 	 tiley 0 part_index 2
9 tilex 1008 	 tiley 0 part_index 2
10 tilex 0 	 tiley 112 part_index 2
11 tilex 112 	 tiley 112 part_index 2
12 tilex 224 	 tiley 112 part_index 2
13 tilex 336 	 tiley 112 part_index 2
14 tilex 448 	 tiley 112 part_index 2
15 tilex 560 	 tiley 112 part_index 2
16 tilex 672 	 tiley 112 part_index 2
17 tilex 784 	 tiley 112 part_index 2
18 tilex 896 	 tiley 112 part_index 2
19 tilex 1008 	 tiley 112 part_index 2
Creati 40 tile-video in total.


In [30]:
subfolder_info[10]

{'folder': '../airmassRGB/supervised/part1_560_0',
 'datetimes': [datetime.datetime(2011, 11, 1, 0, 0),
  datetime.datetime(2011, 11, 1, 0, 5),
  datetime.datetime(2011, 11, 1, 0, 10),
  datetime.datetime(2011, 11, 1, 0, 15),
  datetime.datetime(2011, 11, 1, 0, 20),
  datetime.datetime(2011, 11, 1, 0, 25),
  datetime.datetime(2011, 11, 1, 0, 30),
  datetime.datetime(2011, 11, 1, 0, 35),
  datetime.datetime(2011, 11, 1, 0, 40),
  datetime.datetime(2011, 11, 1, 0, 45),
  datetime.datetime(2011, 11, 1, 0, 50),
  datetime.datetime(2011, 11, 1, 0, 55),
  datetime.datetime(2011, 11, 1, 1, 0),
  datetime.datetime(2011, 11, 1, 1, 5),
  datetime.datetime(2011, 11, 1, 1, 10),
  datetime.datetime(2011, 11, 1, 1, 15)],
 'tile_x': 560,
 'tile_y': 0}

In [31]:
len(subfolder_info)

40

In [1]:
# 2) Carico i ciclonici
tracks_file = "./TRACKS_CL7.dat"  
df_tracks = load_cyclones_track_noheader(tracks_file)

NameError: name 'load_cyclones_track_noheader' is not defined

In [123]:
m = df_tracks.year == 2018 #.unique()
mset = df_tracks[m].month == 9
df_tracks[m&mset]

,id_cyc,lat,lon,time,year,month,day,hour
131456,00001618,8.676,43.439,2018-09-01 00:00:00,2018,9,1,0
131457,00001618,8.943,43.447,2018-09-01 01:00:00,2018,9,1,1
131458,00001618,9.247,43.476,2018-09-01 02:00:00,2018,9,1,2
131459,00001618,9.518,43.518,2018-09-01 03:00:00,2018,9,1,3
131460,00001618,9.719,43.569,2018-09-01 04:00:00,2018,9,1,4
...,...,...,...,...,...,...,...,...
131895,00001622,24.821,38.373,2018-09-30 19:00:00,2018,9,30,19
131896,00001622,24.910,38.392,2018-09-30 20:00:00,2018,9,30,20
131897,00001622,24.998,38.418,2018-09-30 21:00:00,2018,9,30,21
131898,00001622,25.069,38.464,2018-09-30 22:00:00,2018,9,30,22


In [104]:
df_out[df_out.label ==1]

,path,start,end,label,start_time,end_time
22395,../airmassRGB/supervised/part316_336_0,1,16,1,2014-11-10 11:55:00,2014-11-10 13:10:00
22396,../airmassRGB/supervised/part317_336_0,1,16,1,2014-11-10 13:15:00,2014-11-10 14:30:00
23716,../airmassRGB/supervised/part1637_336_0,1,16,1,2020-03-15 23:05:00,2020-03-16 00:20:00
23717,../airmassRGB/supervised/part1638_336_0,1,16,1,2020-03-16 00:25:00,2020-03-16 01:40:00
23718,../airmassRGB/supervised/part1639_336_0,1,16,1,2020-03-16 01:45:00,2020-03-16 03:00:00
...,...,...,...,...,...,...
139242,../airmassRGB/supervised/part6763_896_112,1,16,1,2020-12-25 23:05:00,2020-12-26 00:20:00
139243,../airmassRGB/supervised/part6764_896_112,1,16,1,2020-12-26 00:25:00,2020-12-26 01:40:00
139244,../airmassRGB/supervised/part6765_896_112,1,16,1,2020-12-26 01:45:00,2020-12-26 03:00:00
139245,../airmassRGB/supervised/part6766_896_112,1,16,1,2020-12-26 03:05:00,2020-12-26 04:20:00


In [84]:
df_out = df_out.rename(columns={'folder':'path'})

In [85]:
df_out.to_csv("./df_videotiles_date.csv", index=False)

### Guardo alcuni video da cancellare dal disco per risparmiare spazio

In [128]:
df_out[df_out['path'].str.contains(r"/part43\d+", regex=True)].label.sum()

0

In [129]:
df_filt = df_out[df_out['path'].apply(os.path.isdir)]
print(f"Numero di cartelle esistenti: {len(df_filt)}, righe totali precedenti: {df_out.shape[0]}")

Numero di cartelle esistenti: 126001, righe totali precedenti: 147200


In [130]:
df_filt[df_filt.label==1].shape[0], df_filt[df_filt.label==0].shape[0], round(df_filt.label.sum()/df_filt.shape[0], 3)

(1512, 124489, 0.012)

In [62]:
def balance_df_by_label(df, label_column="label", random_state=478562):
    """
    Bilancia un DataFrame mantenendo tutte le righe con label=1
    e selezionando (in modo random) lo stesso numero di righe con label=0.
    Restituisce un DataFrame mescolato con uguale quantità di 1 e 0.
    
    Parametri:
    - df: DataFrame originale
    - label_column: il nome della colonna che contiene la label (default: 'label')
    - random_state: per riproducibilità del campionamento casuale
    """
    # Seleziona tutte le righe con label=1
    df_pos = df[df[label_column] == 1]
    # Seleziona tutte le righe con label=0
    df_neg = df[df[label_column] == 0]

    # Numero di righe positive
    num_pos = len(df_pos)
    # Esegui un sample sulle righe negative pari a num_pos
    df_neg_sampled = df_neg.sample(n=num_pos, random_state=random_state)

    # Concatena e mescola
    df_balanced = pd.concat([df_pos, df_neg_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [131]:
df_balanced = balance_df_by_label(df_filt)
print(len(df_balanced), df_balanced['label'].value_counts())

3024 label
1    1512
0    1512
Name: count, dtype: int64


In [137]:
df_balanced


,path,start,end,label,start_time,end_time
0,../airmassRGB/supervised/part6762_784_112,1,16,1,2020-12-25 21:45:00,2020-12-25 23:00:00
1,../airmassRGB/supervised/part6102_784_112,1,16,1,2020-11-19 04:50:00,2020-11-19 06:05:00
2,../airmassRGB/supervised/part6101_784_112,1,16,1,2020-11-19 03:30:00,2020-11-19 04:45:00
3,../airmassRGB/supervised/part3246_560_0,1,16,1,2020-06-13 11:20:00,2020-06-13 12:35:00
4,../airmassRGB/supervised/part1925_336_112,1,16,0,2020-04-01 00:05:00,2020-04-01 01:20:00
...,...,...,...,...,...,...
3019,../airmassRGB/supervised/part1653_560_0,1,16,1,2020-03-16 20:25:00,2020-03-16 21:40:00
3020,../airmassRGB/supervised/part1993_896_112,1,16,0,2020-04-04 18:45:00,2020-04-04 20:00:00
3021,../airmassRGB/supervised/part3207_448_112,1,16,1,2020-06-11 07:20:00,2020-06-11 08:35:00
3022,../airmassRGB/supervised/part2788_336_112,1,16,0,2020-05-18 23:05:00,2020-05-19 00:20:00


In [135]:
l = df_balanced.shape[0]
#l = 100
tr_p = int(l * 0.7)
val_p = int(l * 0.9)
print(f"ranges train e validation:  {tr_p} - {val_p}")
df_train = df_balanced[:tr_p]
df_val = df_balanced[tr_p:val_p]
df_test = df_balanced[val_p:]

ranges train e validation:  2116 - 2721


In [136]:
out_csv_label = os.path.join("./", "train_supervised.csv")
df_train.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "val_supervised.csv")
df_val.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "test_supervised.csv")
df_test.to_csv(out_csv_label, index=False)

### copia le immagini in sottocartelle .../partN

In [27]:
from build_supervised_dataset_no_tiles import split_into_subfolders_and_track_dates

In [ ]:
#subfolders = split_into_subfolders(sorted_filenames, output_dir)
subfolder_info = split_into_subfolders_and_track_dates(sorted_filenames, output_dir)

In [ ]:
#subfolder_info

### Suddivide in train/test/val e scrive i CSV

In [28]:
output_dir

'E:/Medicanes_Data/airmassRGB'

In [30]:
from build_dataset import create_csv

In [33]:
create_csv(output_dir)

File CSV generati:
Train: E:/Medicanes_Data/airmassRGB\train.csv
Test: E:/Medicanes_Data/airmassRGB\test.csv
Validation: E:/Medicanes_Data/airmassRGB\val.csv


# Per il dataset supervisionato 
(senza tiles -> sbagliato perché ogni etichetta va associata alla singola tile)

In [ ]:
from build_supervised_dataset_no_tiles import create_supervised_csv_from_info, is_in_medicane
from medicane_utils.load_files import load_medicane_intervals

In [36]:
folder_root = output_dir      # cartella con subfolder part1, part2 ...
medicane_csv = "./medicane_validi.csv"    # le date dei medicane
out_csv = "./dataset.csv"
create_supervised_csv_from_info(subfolder_info, medicane_csv, out_csv)

Creato CSV supervisionato in: ./dataset.csv
